In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
tf.compat.v1.disable_eager_execution()

import os,sys
sys.path.append(os.path.join(os.getcwd(), ".."))

from superpoint.settings import EXPER_PATH
import superpoint.evaluations.detector_evaluation as ev
from utils import plot_imgs

%matplotlib inline
%load_ext autoreload
%autoreload 2

In [ ]:
experiments = [
    'mp_synth-v6_no-aug_synth', 'mp_synth-v6_no-aug_synth-noise',
    'mp_synth-v6_photo-aug_synth', 'mp_synth-v6_photo-aug_synth-noise',
    'harris_synth-v6', 'harris_synth-v6-noise',
    'shi_synth-v6', 'shi_synth-v6-noise',
    'fast_synth-v6', 'fast_synth-v6-noise',
]
confidence_thresholds = [0.1]*4 + [90000]*2 + [0.06]*2 + [40]*2

# PR curve

In [ ]:
plt.figure(dpi=150)
for e, t in zip(experiments, confidence_thresholds):
    p, r, prob = ev.compute_pr(e, distance_thresh=4)
    plt.plot(r, p, label=e)
    
    mAP = ev.compute_mAP(p, r)
    print('> {}'.format(e))
    print('mAP: {}'.format(mAP))
    
    op_idx = np.max(np.where(prob > t))
    plt.plot(r[op_idx], p[op_idx], marker='o', color='k', markersize=3)
    
plt.legend(), plt.xlabel('Recall'), plt.ylabel('Precision')

# Localization Error

In [ ]:
for e, t in zip(experiments, confidence_thresholds):
    error = ev.compute_loc_error(e, prob_thresh=t, distance_thresh=4)
    print('> {}: {}'.format(e, error))

# Visualization

In [ ]:
def draw_keypoints(img, corners, color):
    keypoints = [cv2.KeyPoint(c[1], c[0], 1) for c in np.stack(corners).T]
    return cv2.drawKeypoints(img.astype(np.uint8), keypoints, None, color=color)

In [ ]:
paths = [ev.get_paths(e) for e in experiments]
for i in range(15):
    images = []
    for j, (e, t) in enumerate(zip(experiments, confidence_thresholds)):
        data = np.load(paths[j][i])
        if e == experiments[0]:
            images.append(draw_keypoints(data['image'], np.where(data['keypoint_map']), (0, 255, 0)))
        images.append(draw_keypoints(data['image'], np.where(data['prob_nms'] > t), (0, 255, 0)))
    plot_imgs([im/255. for im in images], titles=['ground truth']+experiments)